In [2]:
import nibabel as nib
import hcp_utils as hcp
import nilearn
import pandas as pd
import numpy as np
from glob import glob
import os
from scipy import stats

In [10]:
# define custom functions
def write_dscalar(data, filename, label=['data']):
    cortL = np.arange(hcp.struct.cortex_left.start,hcp.struct.cortex_left.stop)
    cortL[:] = 1
    cortL = hcp.left_cortex_data(cortL)
    cortR = np.arange(hcp.struct.cortex_right.start,hcp.struct.cortex_right.stop)-hcp.struct.cortex_right.start
    cortR[:] = 1
    cortR = hcp.right_cortex_data(cortR)
    scalar_axis = nib.cifti2.ScalarAxis(label)
    brain_model = nib.cifti2.BrainModelAxis.from_mask(cortL, 'CortexLeft')
    brain_model = brain_model + nib.cifti2.BrainModelAxis.from_mask(cortR, 'CortexRight')
    cifti = nib.Cifti2Image(data, header=nib.Cifti2Header.from_axes((scalar_axis, brain_model)))
    cifti.to_filename(filename)
    
def corr2_coeff(A, B):
    # Rowwise mean of input arrays & subtract from input arrays themeselves
    A_mA = A - A.mean(1)[:, None]
    B_mB = B - B.mean(1)[:, None]

    # Sum of squares across rows
    ssA = (A_mA**2).sum(1)
    ssB = (B_mB**2).sum(1)

    # Finally get corr coeff
    return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))

In [7]:
# setup data/paths
hcp_path = '/arc/project/st-tv01-1/hcp'
data_path = os.path.join(hcp_path, 'data-clean')
out_path = '/scratch/st-tv01-1/hcp/targets/dual_reg'
sub = np.loadtxt(os.path.join(hcp_path,'targets','m2m4_sub_n109.csv'), dtype=str)
if not os.path.isdir(out_path):
    os.makedirs(out_path)

In [8]:
# compute group average seed map for each condition
condition = ['MOVIE2', 'MOVIE4', 'REST1', 'REST4']
sg_mask = hcp.mmp.map_all[hcp.struct.cortex] == 164
group_seedmap = np.zeros((len(condition),len(sg_mask)))
for i,c in enumerate(condition):
    group = np.zeros((0, hcp.struct.cortex.stop))
    n = np.zeros(group.shape[1])
    for s in sub:
        nii = glob(os.path.join(data_path, s + '*','*'+c+'*.nii'))
        if len(nii) == 1:
            cifti = nib.load(nii[0]).get_fdata()[:,hcp.struct.cortex]
            if group.shape[0] == 0:
                group = np.zeros((cifti.shape[0], group.shape[1]))
            mask = ~np.isnan(cifti.mean(axis=0))
            group[:, mask] = group[:, mask] + cifti[:, mask]
            n[mask] = n[mask] + 1
        else:
            print(f'ERROR:\t{s} {c} not found')
    # group average cifti
    group = np.divide(group, n)
    # seed map
    sg_tc = group[:,sg_mask].mean(axis=1).reshape((1,-1))
    group_seedmap[i,:] = np.arctanh(corr2_coeff(group.T, sg_tc))[:,0]
# write all seedmaps to a cifti
write_dscalar(group_seedmap, f'{out_path}/group_seedmap_n{len(sub)}.dscalar.nii', label=condition)
    

pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-

pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-

pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-

NameError: name 'group_cifti' is not defined

In [ ]:
# load group seedmap
group_seedmap = nib.load(f'{out_path}/group_seedmap_n{len(sub)}.dscalar.nii').get_fdata()[:,hcp.struct.cortex]

# compute individual seed maps for each condition
indiv_out = os.path.join(out_path, 'indiv')
if not os.path.isdir(indiv_out):
    os.makedirs(indiv_out)
condition = ['MOVIE2', 'MOVIE4', 'REST1', 'REST4']
for i,c in enumerate(condition):
    group = np.zeros((0, hcp.struct.cortex.stop))
    n = np.zeros(group.shape[1])
    for s in sub:
        nii = glob(os.path.join(data_path, s + '*','*'+c+'*.nii'))
        if len(nii) == 1:
            # load cifti and z-score
            cifti = nib.load(nii[0]).get_fdata()[:,hcp.struct.cortex]
            mask = ~np.isnan(cifti.mean(axis=0))
            cifti = stats.zscore(cifti, axis=0)
            # spatial regression to get seed tc
            seed_tc = np.matmul(cifti, group_seedmap[i,:]).reshape((1,-1))
            indiv_seedmap = np.zeros((1,cifti.shape[1]))
            indiv_seedmap[:] = np.nan
            indiv_seedmap[:,mask] = corr2_coeff(cifti.T, seed_tc)[mask,0]
            # seed map
            write_dscalar(indiv_seedmap, f'{indiv_out}/{c}_{s}_seedmap.dscalar.nii')
        else:
            print(f'ERROR:\t{s} {c} not found')